In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
import plotly.express as px 
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFAutoModelForSequenceClassification
from scipy.special import softmax
import warnings 
warnings.filterwarnings("ignore")

In [28]:
df=pd.read_csv("twitter_training.csv",header=None,names=["id","game","rating","comment"])

In [29]:
df.drop(columns=["id","game"],inplace=True)

In [30]:
df.isnull().sum()
df.dropna(inplace=True)

In [47]:
df["rating"].unique()

array(['Positive', 'Neutral', 'Negative', 'Irrelevant'], dtype=object)

In [31]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [ ]:

import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer

label_map = {"Negative": 0, "Neutral": 1, "Positive": 2, "Irrelevant": 3}

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)

train_texts = train_df['comment'].tolist()
train_labels = [label_map[l] for l in train_df['rating']]

test_texts = test_df['comment'].tolist()
test_labels = [label_map[l] for l in test_df['rating']]

model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)

max_len = 64  
batch_size = 4

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_len, return_tensors='tf')
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=max_len, return_tensors='tf')

train_labels = tf.convert_to_tensor(train_labels, dtype=tf.int32)
test_labels = tf.convert_to_tensor(test_labels, dtype=tf.int32)

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
)).shuffle(1000).batch(batch_size)

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels
)).batch(batch_size)

base_model = TFAutoModel.from_pretrained(model_name)
num_labels = 4

input_ids = tf.keras.layers.Input(shape=(None,), dtype=tf.int32, name="input_ids")
attention_mask = tf.keras.layers.Input(shape=(None,), dtype=tf.int32, name="attention_mask")

outputs = base_model(input_ids, attention_mask=attention_mask)[0]  
logits = tf.keras.layers.Dense(num_labels, activation=None)(cls_token)
model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=logits)

optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

history = model.fit(train_dataset, validation_data=test_dataset, epochs=3)

model.sa("./tf_sentiment_model")
tokenizer.save_pretrained("./tf_sentiment_model")



Some layers from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment were not used when initializing TFRobertaModel: ['classifier']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/3


2025-10-26 20:56:45.061016: I external/local_xla/xla/service/service.cc:163] XLA service 0x75f6ed87b5f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-10-26 20:56:45.061065: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce GTX 1660 SUPER, Compute Capability 7.5
2025-10-26 20:56:46.312552: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-10-26 20:56:47.236802: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91002
I0000 00:00:1761501408.555637   10475 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


14999/14999 [==============================] - 2550s 167ms/step - loss: 0.7739 - sparse_categorical_accuracy: 0.6990 - val_loss: 0.5198 - val_sparse_categorical_accuracy: 0.8173
Epoch 2/3
14999/14999 [==============================] - 2113s 141ms/step - loss: 0.3351 - sparse_categorical_accuracy: 0.8775 - val_loss: 0.3398 - val_sparse_categorical_accuracy: 0.8837
Epoch 3/3
14999/14999 [==============================] - 2076s 138ms/step - loss: 0.1989 - sparse_categorical_accuracy: 0.9258 - val_loss: 0.3527 - val_sparse_categorical_accuracy: 0.8961


AttributeError: 'Functional' object has no attribute 'save_pretrained'

In [51]:
model.save("model.keras")